<a href="https://colab.research.google.com/github/culiacanai/Aprende_Python_con_GoogleColab/blob/main/notebooks/08_Visualizacion_con_Matplotlib.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 📊 Visualización con Matplotlib

### Aprende Python con Google Colab — por [Culiacan.AI](https://culiacan.ai)

**Nivel:** 🟡 Intermedio  
**Duración estimada:** 75 minutos  
**Requisitos:** Haber completado el [Notebook 07 — Pandas Básico](07_Pandas_Basico.ipynb)

---

En este notebook vas a:
- Crear gráficas profesionales con Matplotlib
- Dominar los tipos de gráficas más usados: barras, líneas, scatter, pie, histogramas
- Personalizar colores, etiquetas, títulos y estilos
- Crear subplots (múltiples gráficas en una figura)
- Combinar Pandas + Matplotlib para análisis visual
- Guardar gráficas como imágenes de alta calidad

> 💡 Una buena gráfica vale más que mil filas de datos. La visualización es clave para comunicar hallazgos y tomar decisiones.


---

## 0. Preparación


In [ ]:
import os, urllib.request
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Descargar datos del repositorio
os.makedirs("datos", exist_ok=True)
base_url = "https://raw.githubusercontent.com/culiacanai/Aprende_Python_con_GoogleColab/main/datos"

for archivo in ["ventas.csv", "empleados.csv"]:
    destino = f"datos/{archivo}"
    if not os.path.exists(destino):
        urllib.request.urlretrieve(f"{base_url}/{archivo}", destino)
        print(f"✅ {archivo} descargado")
    else:
        print(f"✅ {archivo} ya existe")

# Cargar datos
df_ventas = pd.read_csv("datos/ventas.csv")
df_ventas["fecha"] = pd.to_datetime(df_ventas["fecha"])
df_ventas["mes"] = df_ventas["fecha"].dt.month

df_empleados = pd.read_csv("datos/empleados.csv")

# Configuración global de Matplotlib
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["figure.dpi"] = 100
plt.rcParams["font.size"] = 12

print(f"\n📊 Datos cargados: {len(df_ventas)} ventas, {len(df_empleados)} empleados")

---

## 1. Tu primera gráfica

Matplotlib funciona con dos conceptos principales:
- **Figure** — el lienzo completo (la imagen)
- **Axes** — el área donde se dibuja la gráfica (puede haber varios en una figura)


In [ ]:
# La forma más simple
ventas_por_suc = df_ventas.groupby("sucursal")["total"].sum().sort_values(ascending=True)

ventas_por_suc.plot(kind="barh")
plt.title("Ventas por Sucursal")
plt.xlabel("Ventas ($)")
plt.show()

In [ ]:
# La forma profesional (más control)
fig, ax = plt.subplots(figsize=(10, 6))

colores = ["#2E86AB" if v >= ventas_por_suc.median() else "#A4C3D2" for v in ventas_por_suc.values]

ax.barh(ventas_por_suc.index, ventas_por_suc.values, color=colores)
ax.set_title("Ventas por Sucursal — Ver de Verdad", fontsize=16, fontweight="bold", pad=15)
ax.set_xlabel("Ventas Totales ($)", fontsize=12)

# Agregar etiquetas de valor en cada barra
for i, v in enumerate(ventas_por_suc.values):
    ax.text(v + 1000, i, f"${v:,.0f}", va="center", fontsize=10)

# Formato del eje X
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x:,.0f}"))

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.tight_layout()
plt.show()

---

## 2. Gráfica de barras

La gráfica más usada para comparar categorías.

### 2.1 Barras verticales


In [ ]:
# Ventas por categoría
ventas_cat = df_ventas.groupby("categoria")["total"].sum().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(8, 5))

colores = ["#2E86AB", "#A23B72", "#F18F01", "#C73E1D"]
barras = ax.bar(ventas_cat.index, ventas_cat.values, color=colores, edgecolor="white", linewidth=1.5)

# Etiquetas encima de cada barra
for barra in barras:
    height = barra.get_height()
    ax.text(barra.get_x() + barra.get_width()/2., height + 2000,
            f"${height:,.0f}", ha="center", va="bottom", fontweight="bold")

ax.set_title("Ventas por Categoría de Producto", fontsize=14, fontweight="bold")
ax.set_ylabel("Ventas ($)")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x:,.0f}"))

plt.tight_layout()
plt.show()

### 2.2 Barras agrupadas


In [ ]:
# Ventas por mes y ciudad
pivot = df_ventas.groupby(["ciudad", "mes"])["total"].sum().unstack(fill_value=0)
meses_nombres = {1: "Enero", 2: "Febrero", 3: "Marzo"}
pivot.columns = [meses_nombres[m] for m in pivot.columns]

fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(pivot.index))
ancho = 0.25
colores = ["#2E86AB", "#F18F01", "#A23B72"]

for i, (mes, color) in enumerate(zip(pivot.columns, colores)):
    barras = ax.bar(x + i * ancho, pivot[mes], ancho, label=mes, color=color, edgecolor="white")

ax.set_title("Ventas por Ciudad y Mes", fontsize=14, fontweight="bold")
ax.set_ylabel("Ventas ($)")
ax.set_xticks(x + ancho)
ax.set_xticklabels(pivot.index)
ax.legend(title="Mes")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x:,.0f}"))

plt.tight_layout()
plt.show()

### 2.3 Barras apiladas


In [ ]:
# Ventas por sucursal y categoría (apiladas)
pivot_suc = df_ventas.groupby(["sucursal", "categoria"])["total"].sum().unstack(fill_value=0)
pivot_suc = pivot_suc.sort_values(by=pivot_suc.columns.tolist(), ascending=False)

fig, ax = plt.subplots(figsize=(12, 6))

colores = ["#2E86AB", "#A23B72", "#F18F01", "#C73E1D"]
pivot_suc.plot(kind="bar", stacked=True, ax=ax, color=colores, edgecolor="white", linewidth=0.5)

ax.set_title("Composición de Ventas por Sucursal", fontsize=14, fontweight="bold")
ax.set_ylabel("Ventas ($)")
ax.set_xlabel("")
ax.legend(title="Categoría", bbox_to_anchor=(1.02, 1), loc="upper left")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.xticks(rotation=45, ha="right")
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x:,.0f}"))

plt.tight_layout()
plt.show()

---

## 3. Gráfica de líneas

Ideal para mostrar tendencias en el tiempo.


In [ ]:
# Tendencia de ventas diarias
ventas_dia = df_ventas.groupby("fecha")["total"].sum()

fig, ax = plt.subplots(figsize=(12, 5))

ax.plot(ventas_dia.index, ventas_dia.values, color="#2E86AB", alpha=0.4, linewidth=1)

# Promedio móvil de 7 días para suavizar
promedio_movil = ventas_dia.rolling(window=7).mean()
ax.plot(promedio_movil.index, promedio_movil.values, color="#2E86AB", linewidth=2.5, label="Promedio 7 días")

ax.set_title("Tendencia de Ventas Diarias", fontsize=14, fontweight="bold")
ax.set_ylabel("Ventas ($)")
ax.legend()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x:,.0f}"))

# Sombrear área bajo la curva
ax.fill_between(ventas_dia.index, ventas_dia.values, alpha=0.1, color="#2E86AB")

plt.tight_layout()
plt.show()

In [ ]:
# Múltiples líneas: tendencia por ciudad
ventas_ciudad_dia = df_ventas.groupby(["fecha", "ciudad"])["total"].sum().unstack(fill_value=0)
ventas_ciudad_semana = ventas_ciudad_dia.resample("W").sum()

fig, ax = plt.subplots(figsize=(12, 6))

colores = {"Culiacán": "#2E86AB", "Mazatlán": "#F18F01", "Los Mochis": "#A23B72", "Guasave": "#C73E1D"}

for ciudad in ventas_ciudad_semana.columns:
    ax.plot(ventas_ciudad_semana.index, ventas_ciudad_semana[ciudad],
            marker="o", markersize=4, linewidth=2, label=ciudad, color=colores.get(ciudad, "gray"))

ax.set_title("Ventas Semanales por Ciudad", fontsize=14, fontweight="bold")
ax.set_ylabel("Ventas ($)")
ax.legend(title="Ciudad")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x:,.0f}"))

plt.tight_layout()
plt.show()

---

## 4. Gráfica de dispersión (scatter)

Muestra la relación entre dos variables numéricas.


In [ ]:
# Relación entre cantidad y total por transacción
fig, ax = plt.subplots(figsize=(10, 6))

# Color por categoría
categorias = df_ventas["categoria"].unique()
colores_cat = {"Lentes": "#2E86AB", "Armazones": "#A23B72", "Accesorios": "#F18F01", "Servicios": "#C73E1D"}

for cat in categorias:
    subset = df_ventas[df_ventas["categoria"] == cat]
    ax.scatter(subset["precio_unitario"], subset["total"],
               alpha=0.5, s=subset["cantidad"] * 30,  # Tamaño según cantidad
               label=cat, color=colores_cat.get(cat, "gray"), edgecolors="white", linewidth=0.5)

ax.set_title("Precio Unitario vs Total de Venta", fontsize=14, fontweight="bold")
ax.set_xlabel("Precio Unitario ($)")
ax.set_ylabel("Total de Venta ($)")
ax.legend(title="Categoría")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.tight_layout()
plt.show()

---

## 5. Gráfica de pastel (pie)

Útil para mostrar proporciones. Úsala solo cuando tengas pocas categorías (máximo 5-6).


In [ ]:
# Distribución de ventas por ciudad
ventas_ciudad = df_ventas.groupby("ciudad")["total"].sum().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(8, 8))

colores = ["#2E86AB", "#F18F01", "#A23B72", "#C73E1D"]
explode = [0.05 if i == 0 else 0 for i in range(len(ventas_ciudad))]

wedges, texts, autotexts = ax.pie(
    ventas_ciudad.values,
    labels=ventas_ciudad.index,
    autopct="%1.1f%%",
    colors=colores,
    explode=explode,
    startangle=90,
    pctdistance=0.85,
    wedgeprops={"edgecolor": "white", "linewidth": 2},
)

# Estilizar texto
for text in autotexts:
    text.set_fontsize(12)
    text.set_fontweight("bold")

# Donut: agregar círculo blanco al centro
centro = plt.Circle((0, 0), 0.65, fc="white")
ax.add_artist(centro)

ax.set_title("Distribución de Ventas por Ciudad", fontsize=14, fontweight="bold", pad=20)

# Texto central
ax.text(0, 0, f"Total\n${ventas_ciudad.sum():,.0f}", ha="center", va="center",
        fontsize=14, fontweight="bold", color="#333333")

plt.tight_layout()
plt.show()

---

## 6. Histograma

Muestra la distribución de una variable numérica.


In [ ]:
# Distribución de montos de venta
fig, ax = plt.subplots(figsize=(10, 5))

ax.hist(df_ventas["total"], bins=30, color="#2E86AB", edgecolor="white", alpha=0.8)

# Líneas de referencia
promedio = df_ventas["total"].mean()
mediana = df_ventas["total"].median()
ax.axvline(promedio, color="#C73E1D", linestyle="--", linewidth=2, label=f"Promedio: ${promedio:,.0f}")
ax.axvline(mediana, color="#F18F01", linestyle="--", linewidth=2, label=f"Mediana: ${mediana:,.0f}")

ax.set_title("Distribución de Montos de Venta", fontsize=14, fontweight="bold")
ax.set_xlabel("Monto ($)")
ax.set_ylabel("Frecuencia")
ax.legend()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Histogramas superpuestos: comparar distribuciones
fig, ax = plt.subplots(figsize=(10, 5))

for ciudad, color in [("Culiacán", "#2E86AB"), ("Mazatlán", "#F18F01")]:
    datos = df_ventas[df_ventas["ciudad"] == ciudad]["total"]
    ax.hist(datos, bins=25, alpha=0.6, color=color, edgecolor="white", label=f"{ciudad} (n={len(datos)})")

ax.set_title("Distribución de Ventas: Culiacán vs Mazatlán", fontsize=14, fontweight="bold")
ax.set_xlabel("Monto ($)")
ax.set_ylabel("Frecuencia")
ax.legend()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.tight_layout()
plt.show()

---

## 7. Box plot (diagrama de caja)

Muestra la distribución, mediana, cuartiles y valores atípicos de una variable.


In [ ]:
# Box plot de ventas por categoría
fig, ax = plt.subplots(figsize=(10, 6))

categorias_orden = df_ventas.groupby("categoria")["total"].median().sort_values(ascending=False).index
datos_box = [df_ventas[df_ventas["categoria"] == cat]["total"].values for cat in categorias_orden]

bp = ax.boxplot(datos_box, labels=categorias_orden, patch_artist=True,
                medianprops={"color": "white", "linewidth": 2},
                whiskerprops={"linewidth": 1.5},
                capprops={"linewidth": 1.5})

colores = ["#2E86AB", "#A23B72", "#F18F01", "#C73E1D"]
for patch, color in zip(bp["boxes"], colores):
    patch.set_facecolor(color)
    patch.set_alpha(0.8)

ax.set_title("Distribución de Ventas por Categoría", fontsize=14, fontweight="bold")
ax.set_ylabel("Monto de Venta ($)")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x:,.0f}"))

plt.tight_layout()
plt.show()

---

## 8. Subplots: múltiples gráficas en una figura

Cuando necesitas mostrar varias perspectivas de los datos juntas:


In [ ]:
# Dashboard con 4 gráficas
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle("Dashboard de Ventas — Ver de Verdad", fontsize=18, fontweight="bold", y=1.02)

# --- 1. Barras: Top sucursales ---
ax1 = axes[0, 0]
top_suc = df_ventas.groupby("sucursal")["total"].sum().sort_values(ascending=True).tail(5)
colores_bar = ["#A4C3D2"] * 4 + ["#2E86AB"]
ax1.barh(top_suc.index, top_suc.values, color=colores_bar)
ax1.set_title("Top 5 Sucursales", fontweight="bold")
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x/1000:.0f}K"))
ax1.spines["top"].set_visible(False)
ax1.spines["right"].set_visible(False)

# --- 2. Líneas: Tendencia mensual ---
ax2 = axes[0, 1]
ventas_semana = df_ventas.groupby(df_ventas["fecha"].dt.isocalendar().week.astype(int))["total"].sum()
ax2.plot(ventas_semana.index, ventas_semana.values, color="#2E86AB", marker="o", markersize=4)
ax2.fill_between(ventas_semana.index, ventas_semana.values, alpha=0.15, color="#2E86AB")
ax2.set_title("Ventas por Semana", fontweight="bold")
ax2.set_xlabel("Semana del año")
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x/1000:.0f}K"))
ax2.spines["top"].set_visible(False)
ax2.spines["right"].set_visible(False)

# --- 3. Pie: Por categoría ---
ax3 = axes[1, 0]
ventas_cat = df_ventas.groupby("categoria")["total"].sum()
colores_pie = ["#2E86AB", "#A23B72", "#F18F01", "#C73E1D"]
wedges, texts, autotexts = ax3.pie(ventas_cat, labels=ventas_cat.index, autopct="%1.0f%%",
                                    colors=colores_pie, startangle=90,
                                    wedgeprops={"edgecolor": "white", "linewidth": 1.5})
for t in autotexts:
    t.set_fontsize(10)
    t.set_fontweight("bold")
ax3.set_title("Ventas por Categoría", fontweight="bold")

# --- 4. Histograma: Distribución de tickets ---
ax4 = axes[1, 1]
ax4.hist(df_ventas["total"], bins=25, color="#2E86AB", edgecolor="white", alpha=0.8)
ax4.axvline(df_ventas["total"].mean(), color="#C73E1D", linestyle="--", linewidth=2,
            label=f"Promedio: ${df_ventas['total'].mean():,.0f}")
ax4.set_title("Distribución de Tickets", fontweight="bold")
ax4.set_xlabel("Monto ($)")
ax4.legend(fontsize=9)
ax4.spines["top"].set_visible(False)
ax4.spines["right"].set_visible(False)

plt.tight_layout()
plt.show()

---

## 9. Estilos y personalización

### 9.1 Estilos predefinidos

Matplotlib incluye varios estilos profesionales:


In [ ]:
# Ver estilos disponibles
print("Estilos disponibles:")
print(plt.style.available)

In [ ]:
# Comparar 4 estilos
estilos = ["default", "seaborn-v0_8", "ggplot", "dark_background"]
datos_ejemplo = df_ventas.groupby("sucursal")["total"].sum().sort_values().tail(5)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for ax, estilo in zip(axes.flat, estilos):
    with plt.style.context(estilo):
        ax.barh(datos_ejemplo.index, datos_ejemplo.values)
        ax.set_title(f'Estilo: "{estilo}"', fontsize=11, fontweight="bold")

plt.tight_layout()
plt.show()

### 9.2 Paleta de colores personalizada

Puedes definir tu propia paleta para mantener consistencia de marca:


In [ ]:
# Paleta de Culiacan.AI / Ver de Verdad
COLORES = {
    "primario": "#2E86AB",
    "secundario": "#A23B72",
    "acento1": "#F18F01",
    "acento2": "#C73E1D",
    "gris_claro": "#E8E8E8",
    "gris_oscuro": "#333333",
    "fondo": "#FAFAFA",
}

PALETA = [COLORES["primario"], COLORES["secundario"], COLORES["acento1"], COLORES["acento2"]]

# Mostrar la paleta
fig, ax = plt.subplots(figsize=(10, 1.5))
for i, (nombre, color) in enumerate(COLORES.items()):
    ax.add_patch(plt.Rectangle((i, 0), 1, 1, color=color))
    ax.text(i + 0.5, 0.5, f"{nombre}\n{color}", ha="center", va="center",
            fontsize=8, color="white" if color in ["#333333", "#2E86AB", "#A23B72", "#C73E1D"] else "black")
ax.set_xlim(0, len(COLORES))
ax.set_ylim(0, 1)
ax.axis("off")
ax.set_title("Paleta de Colores", fontweight="bold")
plt.tight_layout()
plt.show()

---

## 10. Atajos: graficar directo desde Pandas

Pandas tiene métodos `.plot()` integrados que usan Matplotlib por debajo — son más rápidos para exploración:


In [ ]:
# Barras directas desde Pandas
df_ventas.groupby("categoria")["total"].sum().sort_values().plot(
    kind="barh", color=PALETA, title="Ventas por Categoría", figsize=(8, 4)
)
plt.xlabel("Ventas ($)")
plt.tight_layout()
plt.show()

In [ ]:
# Múltiples líneas desde pivot
pivot_mes = df_ventas.groupby([df_ventas["fecha"].dt.to_period("W"), "ciudad"])["total"].sum().unstack(fill_value=0)

pivot_mes.plot(figsize=(12, 5), title="Ventas Semanales por Ciudad", color=PALETA)
plt.ylabel("Ventas ($)")
plt.legend(title="Ciudad")
plt.tight_layout()
plt.show()

In [ ]:
# Análisis rápido de empleados
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribución de sueldos
df_empleados["sueldo"].plot(kind="hist", bins=15, ax=axes[0], color=COLORES["primario"],
                             edgecolor="white", title="Distribución de Sueldos")
axes[0].set_xlabel("Sueldo ($)")

# Empleados por puesto
df_empleados["puesto"].value_counts().plot(kind="bar", ax=axes[1], color=PALETA,
                                            title="Empleados por Puesto", rot=45)
axes[1].set_ylabel("Cantidad")

plt.tight_layout()
plt.show()

---

## 11. Guardar gráficas como imagen


In [ ]:
# Crear una gráfica y guardarla
fig, ax = plt.subplots(figsize=(10, 6))

top_suc = df_ventas.groupby("sucursal")["total"].sum().sort_values(ascending=True)
ax.barh(top_suc.index, top_suc.values, color=COLORES["primario"])
ax.set_title("Ventas por Sucursal — Ver de Verdad", fontsize=16, fontweight="bold")
ax.set_xlabel("Ventas ($)")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x:,.0f}"))

plt.tight_layout()

# Guardar en diferentes formatos
fig.savefig("datos/ventas_sucursal.png", dpi=300, bbox_inches="tight")
fig.savefig("datos/ventas_sucursal.pdf", bbox_inches="tight")

print("✅ Gráfica guardada como PNG (300 DPI) y PDF")
plt.show()

---

## 12. 🏆 Mini Proyecto: Reporte visual ejecutivo

Vamos a crear un reporte de una página con múltiples gráficas:


In [ ]:
# 🏆 Mini Proyecto: Reporte Visual Ejecutivo

fig = plt.figure(figsize=(16, 20))
fig.patch.set_facecolor(COLORES["fondo"])

# Título principal
fig.suptitle("REPORTE EJECUTIVO DE VENTAS\nVer de Verdad — Enero a Marzo 2025",
             fontsize=20, fontweight="bold", color=COLORES["gris_oscuro"], y=0.98)

# --- KPIs (texto en la parte superior) ---
ax_kpi = fig.add_axes([0.05, 0.92, 0.9, 0.04])
ax_kpi.axis("off")

total_ventas = df_ventas["total"].sum()
total_trans = len(df_ventas)
ticket_prom = df_ventas["total"].mean()
mejor_suc = df_ventas.groupby("sucursal")["total"].sum().idxmax()

kpis_text = f"💰 Ventas: ${total_ventas:,.0f}    |    📦 Transacciones: {total_trans}    |    🎫 Ticket Prom: ${ticket_prom:,.0f}    |    🏆 Mejor: {mejor_suc}"
ax_kpi.text(0.5, 0.5, kpis_text, ha="center", va="center", fontsize=13, color=COLORES["gris_oscuro"])

# --- 1. Ventas por sucursal (barras horizontales) ---
ax1 = fig.add_subplot(3, 2, 1)
top = df_ventas.groupby("sucursal")["total"].sum().sort_values(ascending=True)
colores_bar = [COLORES["primario"] if v >= top.median() else "#A4C3D2" for v in top.values]
ax1.barh(top.index, top.values, color=colores_bar)
for i, v in enumerate(top.values):
    ax1.text(v + 500, i, f"${v:,.0f}", va="center", fontsize=8)
ax1.set_title("Ventas por Sucursal", fontweight="bold", fontsize=12)
ax1.spines["top"].set_visible(False)
ax1.spines["right"].set_visible(False)
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x/1000:.0f}K"))

# --- 2. Tendencia semanal ---
ax2 = fig.add_subplot(3, 2, 2)
ventas_semana = df_ventas.groupby(df_ventas["fecha"].dt.isocalendar().week.astype(int))["total"].sum()
ax2.plot(ventas_semana.index, ventas_semana.values, color=COLORES["primario"], marker="o", markersize=4, linewidth=2)
ax2.fill_between(ventas_semana.index, ventas_semana.values, alpha=0.15, color=COLORES["primario"])
ax2.set_title("Tendencia Semanal", fontweight="bold", fontsize=12)
ax2.set_xlabel("Semana")
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x/1000:.0f}K"))
ax2.spines["top"].set_visible(False)
ax2.spines["right"].set_visible(False)

# --- 3. Distribución por categoría (donut) ---
ax3 = fig.add_subplot(3, 2, 3)
ventas_cat = df_ventas.groupby("categoria")["total"].sum().sort_values(ascending=False)
wedges, texts, autotexts = ax3.pie(ventas_cat, labels=ventas_cat.index, autopct="%1.0f%%",
    colors=PALETA, startangle=90, wedgeprops={"edgecolor": "white", "linewidth": 2}, pctdistance=0.8)
for t in autotexts:
    t.set_fontsize(10)
    t.set_fontweight("bold")
centro = plt.Circle((0, 0), 0.6, fc="white")
ax3.add_artist(centro)
ax3.set_title("Ventas por Categoría", fontweight="bold", fontsize=12)

# --- 4. Top 5 productos ---
ax4 = fig.add_subplot(3, 2, 4)
top_prod = df_ventas.groupby("producto")["total"].sum().sort_values(ascending=True).tail(5)
ax4.barh(top_prod.index, top_prod.values, color=COLORES["secundario"])
for i, v in enumerate(top_prod.values):
    ax4.text(v + 500, i, f"${v:,.0f}", va="center", fontsize=8)
ax4.set_title("Top 5 Productos", fontweight="bold", fontsize=12)
ax4.spines["top"].set_visible(False)
ax4.spines["right"].set_visible(False)
ax4.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x/1000:.0f}K"))

# --- 5. Distribución de tickets ---
ax5 = fig.add_subplot(3, 2, 5)
ax5.hist(df_ventas["total"], bins=30, color=COLORES["primario"], edgecolor="white", alpha=0.8)
ax5.axvline(df_ventas["total"].mean(), color=COLORES["acento2"], linestyle="--", linewidth=2,
            label=f"Promedio: ${df_ventas['total'].mean():,.0f}")
ax5.axvline(df_ventas["total"].median(), color=COLORES["acento1"], linestyle="--", linewidth=2,
            label=f"Mediana: ${df_ventas['total'].median():,.0f}")
ax5.set_title("Distribución de Tickets", fontweight="bold", fontsize=12)
ax5.set_xlabel("Monto ($)")
ax5.legend(fontsize=9)
ax5.spines["top"].set_visible(False)
ax5.spines["right"].set_visible(False)

# --- 6. Ventas por ciudad y mes ---
ax6 = fig.add_subplot(3, 2, 6)
pivot_cm = df_ventas.groupby(["ciudad", "mes"])["total"].sum().unstack(fill_value=0)
meses_n = {1: "Ene", 2: "Feb", 3: "Mar"}
pivot_cm.columns = [meses_n[m] for m in pivot_cm.columns]
pivot_cm.plot(kind="bar", ax=ax6, color=[COLORES["primario"], COLORES["acento1"], COLORES["secundario"]],
              edgecolor="white", linewidth=0.5)
ax6.set_title("Ventas por Ciudad y Mes", fontweight="bold", fontsize=12)
ax6.set_xlabel("")
ax6.legend(title="Mes", fontsize=9)
plt.setp(ax6.get_xticklabels(), rotation=45, ha="right")
ax6.spines["top"].set_visible(False)
ax6.spines["right"].set_visible(False)
ax6.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x/1000:.0f}K"))

# Pie de página
fig.text(0.5, 0.01, "Generado con Python + Matplotlib | Culiacan.AI", ha="center",
         fontsize=10, color="gray", style="italic")

plt.tight_layout(rect=[0, 0.02, 1, 0.93])
fig.savefig("datos/reporte_ejecutivo.png", dpi=200, bbox_inches="tight", facecolor=COLORES["fondo"])
print("✅ Reporte guardado como datos/reporte_ejecutivo.png")
plt.show()

---

## 🔥 Retos

1. **Heatmap de ventas:** Crea un heatmap (mapa de calor) que muestre las ventas por sucursal (filas) y día de la semana (columnas). Usa `ax.imshow()` o `ax.pcolormesh()`. ¿Qué día vende más cada sucursal?

2. **Gráfica de cascada (waterfall):** Muestra cómo se compone el total de ventas por categoría, empezando desde 0 y acumulando cada categoría hasta llegar al total.

3. **Análisis de nómina visual:** Crea un dashboard de 4 gráficas para los empleados: distribución de sueldos (histograma), sueldo por puesto (box plot), empleados por sucursal (barras), y nómina total por ciudad (pie).


In [ ]:
# Reto 1: Heatmap de ventas
# Tu código aquí 👇


In [ ]:
# Reto 2: Gráfica de cascada
# Tu código aquí 👇


In [ ]:
# Reto 3: Dashboard de nómina
# Tu código aquí 👇


---

## 📋 Resumen

### Tipos de gráficas

| Tipo | Cuándo usarla | Código |
|------|-------------|--------|
| **Barras** | Comparar categorías | `ax.bar()` / `ax.barh()` |
| **Líneas** | Tendencias en el tiempo | `ax.plot()` |
| **Scatter** | Relación entre 2 variables | `ax.scatter()` |
| **Pie/Donut** | Proporciones (pocas categorías) | `ax.pie()` |
| **Histograma** | Distribución de una variable | `ax.hist()` |
| **Box plot** | Distribución + outliers | `ax.boxplot()` |

### Personalización

| Qué | Código |
|-----|--------|
| Título | `ax.set_title("Título", fontweight="bold")` |
| Etiquetas | `ax.set_xlabel()`, `ax.set_ylabel()` |
| Leyenda | `ax.legend()` |
| Quitar bordes | `ax.spines["top"].set_visible(False)` |
| Formato numérico | `ax.yaxis.set_major_formatter(...)` |
| Subplots | `fig, axes = plt.subplots(filas, columnas)` |
| Guardar | `fig.savefig("archivo.png", dpi=300)` |
| Desde Pandas | `df.plot(kind="bar")` |

---

## ⏭️ ¿Qué sigue?

En el siguiente notebook aprenderás **Web Scraping** — cómo extraer datos de páginas web automáticamente con Python.

👉 [09 — Web Scraping Básico](09_Web_Scraping_Basico.ipynb)

---

<p align="center">
  Hecho con ❤️ por <a href="https://culiacan.ai">Culiacan.AI</a> — Culiacán reconocida en el mundo por su talento y emprendimiento en Inteligencia Artificial
</p>
